In [9]:
# print('dataset.x' ,dataset.x)
# print('dataset.edge_index' ,dataset.edge_index)
# print('dataset.edge_attr' ,dataset.edge_attr)
%load_ext autoreload
%autoreload 2
import sys
import torch

sys.path.insert(0,'/home/aw1223/ip/agile')

from sdk.ample import Ample

from torch_geometric.datasets import FakeDataset #TODO remove
from sdk.models.models import MLP_Model,Interaction_Net_Model

from torch_geometric.data import Data



class ToyModel(torch.nn.Module):
    def __init__(self, in_channels=32, out_channels=32, layer_count=1, hidden_dimension=32, precision = torch.float32):
        super().__init__()
        self.precision = precision
        self.layers = torch.nn.ModuleList()


        self.linear1 = MLP_Model(in_channels, out_channels) 
        self.linear1.name  = 'linear_embedder1'
        self.layers.append(self.linear1) #Used to map weights in SDK


        self.linear2 = MLP_Model(in_channels, out_channels) 
        self.linear2.name  = 'linear_embedder2'
        self.layers.append(self.linear2) #Used to map weights in SDK

        # self.int_net = Interaction_Net_Model(in_channels, out_channels, layer_count, hidden_dimension, precision)
        # self.int_net.name  = 'int_net'
        # self.layers.append(self.int_net) #Used to map weights in SDK


        for layer in self.layers:
            layer.to(self.precision)

    def forward(self, x):
        outputs_model = []
        x = x.to(self.precision) 
        print(x) 
        outputs_sub_model1,x = self.linear1(x)
        print('outputs_sub_model lin 1')
        print(outputs_sub_model1)
        #need to add a name to each model output as order of computation is not guaranteed
        # outputs_model = outputs_model + outputs_sub_model #Add instead of append to have layer outputs in single list as tb iterates over each layer
        # print(x)

        outputs_sub_model2,x = self.linear2(x)
        print('outputs_sub_model lin 2')
        print(outputs_sub_model2)
        outputs_model = outputs_sub_model1 + outputs_sub_model2
        # _,x = self.int_net(x, edge_index, edge_attr)

        return outputs_model,x

model = ToyModel(32,32)

dataset = Data()
dataset.x = torch.tensor([[-2.0775e-01,  5.4138e-01, -9.3228e-01, -1.5647e-02, -9.2422e-01,
         -1.4551e+00,  1.1447e+00,  4.8944e-01, -1.2190e+00, -2.1434e+00,
          8.0335e-01, -1.3588e+00, -1.2911e+00,  1.1237e+00,  1.1250e-01,
          3.9626e-01,  5.1411e-01,  9.9543e-01,  7.6991e-02,  1.1795e+00,
         -1.2423e+00, -2.6467e-01, -4.9839e-01, -1.0298e+00, -2.2073e+00,
          1.7572e+00, -5.1693e-01,  1.4884e+00,  1.1717e+00, -1.6047e+00,
         -1.0268e+00,  1.6982e+00],
        [-1.8529e+00,  1.6048e+00, -6.7576e-01,  4.8206e-01, -7.3550e-01,
          2.7671e+00,  6.5742e-01,  1.1275e-01, -8.2426e-01,  9.4348e-01,
         -5.4252e-01,  1.3881e+00, -3.2231e-01,  2.2466e+00,  2.9660e-01,
         -3.3892e-01, -3.5646e-01,  1.7096e+00,  6.5559e-01,  9.3671e-01,
         -2.0564e-01, -2.5100e-01, -6.0347e-02, -9.6708e-01,  2.2658e+00,
          2.2228e-01, -2.8040e+00, -8.2614e-01,  5.2462e-01, -2.1695e+00,
         -1.4330e-02,  8.4461e-01],
        [ 6.7803e+00,  5.0186e+00,  5.3118e+00,  6.6033e+00,  5.3946e+00,
          4.0241e+00,  5.3939e+00,  5.0030e+00,  4.8918e+00,  3.9511e+00,
          6.0363e+00,  4.9824e+00,  5.2248e+00,  6.5227e+00,  5.0742e+00,
          4.1273e+00,  4.4227e+00,  5.2284e+00,  5.5740e+00,  6.3545e+00,
          4.3107e+00,  5.2994e+00,  5.3288e+00,  5.3148e+00,  5.8875e+00,
          5.0108e+00,  5.8182e+00,  4.4662e+00,  5.2387e+00,  6.0119e+00,
          4.5336e+00,  5.5987e+00]])
dataset.edge_index = torch.tensor([[ 0,  0,  0,  1,  1,  1, 2,  2,  2],
        [ 0,  1, 2,  0,  1,  2,  0,  1, 2]])
dataset.edge_attr =  None


inputs = [dataset.x]
out = model(*inputs)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
tensor([[-0.2077,  0.5414, -0.9323, -0.0156, -0.9242, -1.4551,  1.1447,  0.4894,
         -1.2190, -2.1434,  0.8033, -1.3588, -1.2911,  1.1237,  0.1125,  0.3963,
          0.5141,  0.9954,  0.0770,  1.1795, -1.2423, -0.2647, -0.4984, -1.0298,
         -2.2073,  1.7572, -0.5169,  1.4884,  1.1717, -1.6047, -1.0268,  1.6982],
        [-1.8529,  1.6048, -0.6758,  0.4821, -0.7355,  2.7671,  0.6574,  0.1128,
         -0.8243,  0.9435, -0.5425,  1.3881, -0.3223,  2.2466,  0.2966, -0.3389,
         -0.3565,  1.7096,  0.6556,  0.9367, -0.2056, -0.2510, -0.0603, -0.9671,
          2.2658,  0.2223, -2.8040, -0.8261,  0.5246, -2.1695, -0.0143,  0.8446],
        [ 6.7803,  5.0186,  5.3118,  6.6033,  5.3946,  4.0241,  5.3939,  5.0030,
          4.8918,  3.9511,  6.0363,  4.9824,  5.2248,  6.5227,  5.0742,  4.1273,
          4.4227,  5.2284,  5.5740,  6.3545,  4.3107,  5.2994,  5.3288,  5.3148,
          5.8875,  

In [10]:
import torch.nn.init as init


print('weights')
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)


%load_ext autoreload
%autoreload 2
%pwd
ample = Ample()
ample.sim = True
#Need weights to be initialized before calling to_device
# model = GraphLam_Model(<parameters>)
model.to_device('ample',data=inputs) #Change 

# out = model(*inputs)

weights
layers.0.layers.0.weight tensor([[-0.0084,  0.0304,  0.1117,  ..., -0.0922, -0.1008, -0.1613],
        [ 0.0309,  0.0559,  0.0508,  ...,  0.0325,  0.0554, -0.1621],
        [ 0.0070,  0.0611,  0.0005,  ..., -0.1199, -0.1162, -0.1191],
        ...,
        [-0.0395,  0.0118, -0.0175,  ..., -0.0227,  0.0451, -0.0544],
        [-0.0576,  0.0287, -0.0501,  ..., -0.0898,  0.1367,  0.1310],
        [-0.0451, -0.0412, -0.0661,  ..., -0.0585,  0.0148, -0.1411]])
layers.1.layers.0.weight tensor([[ 0.0710,  0.0266,  0.0160,  ..., -0.0566, -0.1146, -0.0305],
        [ 0.1374, -0.1612, -0.1360,  ..., -0.1006, -0.0068,  0.0423],
        [-0.1172, -0.0600, -0.0220,  ...,  0.1420,  0.1664,  0.1044],
        ...,
        [-0.0066,  0.0116, -0.1186,  ..., -0.1546, -0.1020, -0.1366],
        [ 0.1754,  0.1621,  0.0796,  ..., -0.0564,  0.0948, -0.0879],
        [ 0.1100,  0.1128, -0.0487,  ...,  0.1129,  0.0227, -0.0095]])
The autoreload extension is already loaded. To reload it, use:
  %reload_e

10it [00:00, 46192.78it/s]
10it [00:00, 59074.70it/s]


Updated configuration successfully written back to file.
saving ToyModel(
  (layers): ModuleList(
    (0-1): 2 x MLP_Model(
      (layers): ModuleList(
        (0): Linear(in_features=32, out_features=32, bias=False)
      )
    )
  )
  (linear1): MLP_Model(
    (layers): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=False)
    )
  )
  (linear2): MLP_Model(
    (layers): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=False)
    )
  )
)
tensor([[-0.2077,  0.5414, -0.9323, -0.0156, -0.9242, -1.4551,  1.1447,  0.4894,
         -1.2190, -2.1434,  0.8033, -1.3588, -1.2911,  1.1237,  0.1125,  0.3963,
          0.5141,  0.9954,  0.0770,  1.1795, -1.2423, -0.2647, -0.4984, -1.0298,
         -2.2073,  1.7572, -0.5169,  1.4884,  1.1717, -1.6047, -1.0268,  1.6982],
        [-1.8529,  1.6048, -0.6758,  0.4821, -0.7355,  2.7671,  0.6574,  0.1128,
         -0.8243,  0.9435, -0.5425,  1.3881, -0.3223,  2.2466,  0.2966, -0.3389,
         -0.3565,  1.7096,  0.

In [11]:
out = model(*inputs)

Executing on AMPLE
cd /home/aw1223/ip/agile/hw/sim
==== Running command: cd /home/aw1223/ip/agile/hw/sim; make run_sim
make -f Makefile.modelsim run_sim
make[1]: Entering directory '/home/aw1223/ip/agile/hw/sim'
set -o pipefail; LIBPYTHON_LOC=/home/aw1223/anaconda3/envs/ample/lib/libpython3.11.so.1.0 MODULE=runner TESTCASE= TOPLEVEL="work.top_wrapper" \
GPI_EXTRA= TOPLEVEL_LANG=verilog \
 /mnt/applications/mentor/modelsim-2019.2/modelsim/modeltech/linux_x86_64/vsim -c -64 -l sim_build/modelsim.log -onfinish exit -do sim_build/../simulate_no_gui.do -quiet -suppress vsim-3015  -pli /home/aw1223/anaconda3/envs/ample/lib/python3.11/site-packages/cocotb/libs/libcocotbvpi_modelsim.so \
 \
work.work_opt \
 2>&1 | tee sim_build/simulate.log
/mnt/applications/mentor/modelsim-2019.2/modelsim/modeltech/linux_x86_64/vish: /mnt/applications/Xilinx/19.2/Vitis/2019.2/lib/lnx64.o/libxml2.so.2: no version information available (required by /lib64/libfontconfig.so.1)
/mnt/applications/mentor/modelsim-20

make[1]: *** [Makefile.modelsim:224: run_sim] Error 211
make: *** [Makefile:11: run_sim] Error 2


In [12]:
import os


def get_expected_outputs(model,data):
  ####Remove bias from the model, TODO Add biases####
  state_dict = model.state_dict()
  for name, param in state_dict.items():
      print('model')
      print(name,param,'name,param')
      if 'bias' in name:
          # Reset the bias tensor to all zeros
          state_dict[name] = torch.zeros_like(param)

  model.load_state_dict(state_dict)
  

  model_input = data
  # if edge_attr is not None:
  #     model_input = model_input + (edge_attr,)  

  model.eval()
  with torch.no_grad():
      # if edge_attr is not None:
      outputs,x = model(*model_input)
      # else:
      #     output = model(x, edge_index)
      # output = model(x, edge_index, edge_attr) if edge_attr is not None else model(x, edge_index)
      # a = b
  del model

  return outputs

def load_jit_model():
  config_path =  os.environ.get("WORKAREA") + "/hw/sim/layer_config/"

  model_path = config_path + 'model.pt'
  print('loading model from',model_path)

  model = torch.jit.load(model_path)
  return model



def load_graph():
  config_path =  os.environ.get("WORKAREA") + "/hw/sim/layer_config/"

  graph_path = config_path + 'graph.pth'
  print('loading graph from',graph_path)

  graph = torch.load(graph_path)
  input_data = graph['input_data']
  # x_loaded = input_data['x']
  # edge_index_loaded = input_data['edge_index']

  # # Check if edge attributes are present and load them if they are
  # edge_attr_loaded = input_data.get('edge_attr', None)
  
  return input_data #(x_loaded, edge_index_loaded, edge_attr_loaded)
  

inputs = load_graph()
model = load_jit_model()
expected_outputs = get_expected_outputs(model,inputs)
# expected_outputs

loading graph from /home/aw1223/ip/agile/hw/sim/layer_config/graph.pth
loading model from /home/aw1223/ip/agile/hw/sim/layer_config/model.pt
model
layers.0.layers.0.weight tensor([[-0.0084,  0.0304,  0.1117,  ..., -0.0922, -0.1008, -0.1613],
        [ 0.0309,  0.0559,  0.0508,  ...,  0.0325,  0.0554, -0.1621],
        [ 0.0070,  0.0611,  0.0005,  ..., -0.1199, -0.1162, -0.1191],
        ...,
        [-0.0395,  0.0118, -0.0175,  ..., -0.0227,  0.0451, -0.0544],
        [-0.0576,  0.0287, -0.0501,  ..., -0.0898,  0.1367,  0.1310],
        [-0.0451, -0.0412, -0.0661,  ..., -0.0585,  0.0148, -0.1411]]) name,param
model
layers.1.layers.0.weight tensor([[ 0.0710,  0.0266,  0.0160,  ..., -0.0566, -0.1146, -0.0305],
        [ 0.1374, -0.1612, -0.1360,  ..., -0.1006, -0.0068,  0.0423],
        [-0.1172, -0.0600, -0.0220,  ...,  0.1420,  0.1664,  0.1044],
        ...,
        [-0.0066,  0.0116, -0.1186,  ..., -0.1546, -0.1020, -0.1366],
        [ 0.1754,  0.1621,  0.0796,  ..., -0.0564,  0.0948,